In [1]:
import os, re, copy
from pathlib import Path
import pandas as pd
import requests 
from stop_words import get_stop_words
import warnings 
from pathlib import Path
from bs4 import BeautifulSoup 
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [2]:
import os
import re
from pprint import pprint
import argparse
import ast
from pathlib import Path
import pathlib
import re
import pprint 
import argparse

p = Path('.')

json_string='''@article{moskowitz1999industries,
  title={Do industries explain momentum?},
  author={Moskowitz, Tobias J and Grinblatt, Mark},
  journal={The Journal of finance},
  volume={54},
  number={4},
  pages={1249--1290},
  year={1999},
  publisher={Wiley Online Library}
}'''

url= f'https://scholar.google.com/scholar?cites=7674247067610031901&as_sdt=5,47&sciodt=0,47&hl=en' # user manual input

collect_dict_values= {}
for item in json_string.split('\n'):
    ii= item.split('=')
    if 'year' in [i.strip() for i in ii ]:
        collect_dict_values['year']=re.sub('\W+','-', item.split('=')[1])
    if 'title' in [i.strip() for i in ii ]:
        collect_dict_values['title']=re.sub('\W+','-', item.split('=')[1])
    if 'author' in [i.strip() for i in ii ]:
        collect_dict_values['author']=re.sub('\W+','-', item.split('=')[1])
        # print(re.sub('\W+','-', item.split('=')[1]))
    if 'journal' in [i.strip() for i in ii ]:
        collect_dict_values['journal']=re.sub('\W+','-', item.split('=')[1])
        
print(''.join(('article-',collect_dict_values['year'],collect_dict_values['author'],
               collect_dict_values['title'], collect_dict_values['journal'][:-1],'.pdf',)))
citation_key= [item.split(',') for item in json_string.split('\n')][0][0].split('{')[1]

print('CITATION KEY: ', citation_key)

citation_key =f'{citation_key}' # user manual input

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
page_contents=requests.get(url,headers=headers)
soup=BeautifulSoup(page_contents.text,'html.parser')

citation_dict={}
def cited_by_gscholar(dsoup):
    cited_by_link_list=[]
    for item in dsoup.find_all('div',{"class" : "gs_ri"}):
        cited_by_link_dict={}
        authors=[]
        for el in item.extract().find_all('a'):
            if 'https' in el.get('href').split(':'):
                if len(set(['View as HTML', 'Cached']) & set([str(el.text)])) ==0:
                    cited_by_link_dict['article_title']=el.text
                    cited_by_link_dict['article_link']=el.get('href')
            if '/citations?user' in el.get('href'):
                authors.append(el.text)
            elhref = el.get('href')
            if '/scholar?cites' in elhref.split('='):
                cited_by_link_dict['cited_by']=elhref
            if 'by' in el.text.split():
                #print(el.text.split())
                citation_cnt=re.findall(r'\d+', el.text)[0]
                cited_by_link_dict['citations_count']=citation_cnt
                cited_by_link_list.append(cited_by_link_dict)
        cited_by_link_dict['auhtors']=authors
    return cited_by_link_list

dsoup = copy.deepcopy(soup)
dsoup= cited_by_gscholar(dsoup)
citation_dict['citation_key']=citation_key
citation_dict['google_scholar_link']=url
citation_dict['cited_by']=dsoup
df = pd.DataFrame.from_dict(citation_dict)
df.to_csv('academic_citation_network.csv', mode='a')

!clear
!git status
!export citation_key=($citation_key)
!git add .
!git commit -m "$citation_key"
!git push

citation_dict

article--1998--Datar-Vinay-T-and-Naik-Narayan-Y-and-Radcliffe-Robert--Liquidity-and-stock-returns-An-alternative-test--Journal-of-financial-markets.pdf
CITATION KEY:  datar1998liquidity
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   academic_citation_network.csv
	modified:   academic_citation_networks_google_scholar.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
[main 7b74a41] datar1998liquidity
 2 files changed, 85 insertions(+), 2 deletions(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 16 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 3.12 KiB | 1.04 MiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To ht

{'citation_key': 'datar1998liquidity',
 'google_scholar_link': 'https://scholar.google.com/scholar?start=30&hl=en&as_sdt=5,47&sciodt=0,47&cites=12007413577990633696&scipsc=',
 'cited_by': [{'article_title': 'Liquidity and asset prices',
   'article_link': 'https://www.nowpublishers.com/article/Details/FIN-003',
   'cited_by': '/scholar?cites=9445589217400078285&as_sdt=5,47&sciodt=0,47&hl=en',
   'citations_count': '1195',
   'auhtors': ['Y Amihud']},
  {'article_title': 'Trading volume and cross‐autocorrelations in stock returns',
   'article_link': 'https://onlinelibrary.wiley.com/doi/abs/10.1111/0022-1082.00231',
   'cited_by': '/scholar?cites=4315853167916969684&as_sdt=5,47&sciodt=0,47&hl=en',
   'citations_count': '1061',
   'auhtors': ['T Chordia', 'B Swaminathan']},
  {'article_title': 'The value of active mutual fund management: An examination of the stockholdings and trades of fund managers',
   'article_link': 'https://www.cambridge.org/core/journals/journal-of-financial-and-q